<a href="https://colab.research.google.com/github/ychoi-kr/llm-api-prog/blob/main/7_langchain/langchain_runnable_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 언어 감지

In [1]:
!pip install fasttext-wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 9.8 MB/s eta 0:00:00


In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

--2024-06-25 19:51:34--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.44, 18.238.176.126, 18.238.176.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 938013 (916K) [binary/octet-stream]
Saving to: ‘lid.176.ftz’

lid.176.ftz         100%[===================>] 916.03K  --.-KB/s    in 0.08s   

2024-06-25 19:51:34 (11.3 MB/s) - ‘lid.176.ftz’ saved [938013/938013]



In [3]:
import fasttext

In [4]:
def detect_language(text):
    model = fasttext.load_model('lid.176.ftz')
    predictions = model.predict(text, k=1)
    lang = predictions[0][0].split('__')[-1]
    return lang

In [5]:
detect_language("안녕하세요")

'ko'

## 다국어 리뷰 감성 분석

In [6]:
!pip install langchain langchain-openai langchain-upstage langchain-community openai pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from langchain_openai import ChatOpenAI
from langchain_upstage import ChatUpstage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import pandas as pd

In [8]:
from google.colab import userdata
import os
os.environ["UPSTAGE_API_KEY"] = userdata.get('UPSTAGE_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [9]:
# ChatOpenAI 설정
chat_openai = ChatOpenAI()

# 다국어 -> 영어 번역 (ChatOpenAI 사용)
translate_to_en_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to English. If it's already in English, return it as is: {text}"
)
translate_to_en_chain = translate_to_en_prompt | chat_openai | StrOutputParser()

In [10]:
# Solar 영한 번역 모델 설정
translator_enko = ChatUpstage(model_name="solar-1-mini-translate-enko")

# 영어 -> 한국어 번역 (Solar 모델 사용)
def translate_to_ko(text):
    return translator_enko.invoke(text).content

In [11]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import json

# Pydantic 모델 정의
class SentimentAnalysis(BaseModel):
    overall_sentiment: str = Field(description="The overall sentiment of the review (Very Positive, Positive, Negative, Very Negative, or Neutral)")
    key_points: List[str] = Field(description="List of key points extracted from the review")

# Pydantic 출력 파서 생성
parser = PydanticOutputParser(pydantic_object=SentimentAnalysis)

# 키포인트 추출 및 감성 분석 프롬프트 수정
extract_points_sentiment_prompt = ChatPromptTemplate.from_template(
    """You are analyzing a review for the 'Book Creator Guide' GPT model. Your task is to extract key points from the given review text and determine the overall sentiment.

    Review: {text}

    Instructions:
    1. Determine the overall sentiment of the review (Positive, Negative, or Neutral).
    2. Extract up to 3 key points from the review that align with this overall sentiment.
    3. Each point must be directly derived from the review text and should reflect the tone and sentiment of the original review.
    4. If the review is very short or lacks detail, it's okay to extract fewer than 3 points.
    5. If you can't find any clear points, provide a single point stating "No specific points could be extracted from this short review."

    {format_instructions}

    Ensure that your response is a valid JSON object with 'overall_sentiment' and 'key_points' fields.

    Analysis:"""
)

# 체인 구성
extract_points_sentiment_chain = extract_points_sentiment_prompt | chat_openai | parser

In [12]:
# 리뷰 리스트
reviews = [
    "This is FANTASTICO! I've wanted to write books my entire life, but lack the executive functioning skills to ever know where to begin. This AI book creator does all the things my ADHD brain can't and all I have to do is punch in the ideas.",
    "fluixet en la representación d'imatges",
    "Muadili diğer uygulamalar ile kıyaslanamayacak kadar güzel. Lütfen Microsoft un bu uygulamanın içine sıçmasına izin vermeyin, teşekkürler",
    "buono il risultato ma la storia dovrebbe essere maggiormente dettagliata",
    "j'adore",
    "感觉还是不行",
    "świetne",
    "no logic. no consistency. confused very easily.",
    "가톨릭에서는 마리아와 성인을 숭배하는 것이 아니라 신앙의 모범으로 공경하고 있습니다. 한국어로 숭배하다라고 해석하는 것은 신으로 숭배하는 것으로 오해를 불러일으킬 수 있는 번역입니다. 따라서 공경하다로 수정하여야 합니다.",
]

In [15]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# 언어 감지 Runnable
detect_language_runnable = RunnableLambda(lambda x: detect_language(x["text"]))

# 영어로 번역 Runnable
translate_to_en_runnable = RunnableLambda(lambda x:
    translate_to_en_chain.invoke({"text": x["text"]}) if x["lang"] != "en" else x["text"])

# 감성 분석 및 키포인트 추출 Runnable
analyze_sentiment_runnable = RunnableLambda(lambda x:
    extract_points_sentiment_chain.invoke({"text": x["en_text"], "format_instructions": parser.get_format_instructions()}))

# 한국어로 번역 Runnable
translate_to_ko_runnable = RunnableLambda(lambda x: {
    "ko_sentiment": translate_to_ko(x["analysis"].overall_sentiment),
    "ko_points": [translate_to_ko(point) for point in x["analysis"].key_points],
    "ko_review": translate_to_ko(x["en_text"]) if x["lang"] != "ko" else x["text"]
})

# 결과 포맷팅 Runnable
format_result_runnable = RunnableLambda(lambda x: {
    "원문": x["text"],
    "감지된 언어": x["lang"],
    "한국어 리뷰": x["ko_results"]["ko_review"],
    "전체 감성": x["ko_results"]["ko_sentiment"],
    "주요 포인트": x["ko_results"]["ko_points"]
})

# 전체 워크플로우 구성
workflow = (
    RunnablePassthrough.assign(lang=detect_language_runnable)
    | RunnablePassthrough.assign(en_text=translate_to_en_runnable)
    | RunnablePassthrough.assign(analysis=analyze_sentiment_runnable)
    | RunnablePassthrough.assign(ko_results=translate_to_ko_runnable)
    | format_result_runnable
)

# 워크플로우 실행
results = workflow.batch(
    [{"text": review} for review in reviews]
)

df = pd.DataFrame(results)

In [16]:
df

,원문,감지된 언어,한국어 리뷰,전체 감성,주요 포인트
0,This is FANTASTICO! I've wanted to write books...,en,"이건 환상적이네요! 평생 책을 쓰고 싶었지만, 어디서부터 시작해야 할지 알 수 없는...",긍정적,[ADHD로 인해 부족한 실행 기능 능력을 검토자가 AI 책 제작기로 도움을 받습니...
1,fluixet en la representación d'imatges,ca,그것은 이미지의 재현에서 흘러나온 것이다.,긍정적,"[이미지의 표현이 잘 흘러갔어요, 책 제작가이드에는 이미지를 효과적으로 활용, 리뷰..."
2,Muadili diğer uygulamalar ile kıyaslanamayacak...,tr,다른 응용 프로그램과 동등하게 비교할 수 없는 아름다운. 제발 마이크로 소프트가 이...,긍정,"[응용 프로그램은 다른 응용 프로그램과 동등한 아름다운, 리뷰어는 응용 프로그램을 ..."
3,buono il risultato ma la storia dovrebbe esser...,it,결과는 좋지만 스토리가 좀 더 디테일하게 나왔으면 좋겠어요,중립,"[결과가 좋으니까요, 스토리가 더 디테일해져야 될 것]"
4,j'adore,fr,내가 애정하는,긍정적,"[리뷰어는 '북크리에이터 가이드'를 아주 좋아한다, 가이드에 대한 긍정적인 경험의 ..."
5,感觉还是不行,ja,여전히 기분이 좋지 않다.,부정적,"[기분이 영 별로다, 리뷰어는 북크리에이터 가이드에 대해 불만족스러워합니다., 부정..."
6,świetne,pl,훌륭함,긍정적.,"[리뷰에서는 책 만들기 가이드를 ‘훌륭하다’고 표현했다., 책제작 가이드가 도움이 ..."
7,no logic. no consistency. confused very easily.,en,논리가 없다. 일관성이 없다. 아주 쉽게 혼란스러워진다.,부정적,"[책 만들기 가이드에 논리가 없어, 내용에 일관성이 없다, 가이드를 사용하면서 쉽게..."
8,가톨릭에서는 마리아와 성인을 숭배하는 것이 아니라 신앙의 모범으로 공경하고 있습니다...,ko,가톨릭에서는 마리아와 성인을 숭배하는 것이 아니라 신앙의 모범으로 공경하고 있습니다...,중립,"[한국어로 마리아와 성인을 공경하는 것을 다시 말하자면 우상숭배로 오해받을 여지, ..."
